In [53]:
'''
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/영상처리/tt.jpg ./
'''
### 이미지는 tt.jpg 형태로 현재폴더에 위치해야함

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import cv2

### 거리측정 메소드 ###
def euclidean(x,center):
    return ((x - center)**2).sum(axis=1)
    
def mahalanobis(x, center):
    dim = len(x)
    mean = np.mean(x, axis=0)
    variance = (x-mean).T.dot((x-mean))/dim
    inverse_variance = np.linalg.inv(variance)
    result = (x-center).dot(inverse_variance).dot((x-center).T)
    return np.diag(result,k=0)

### 초기화 메소드 ###
def random_initialize(k, data, dim):
    centers = []
    for i in range(dim):
      centers.append(np.random.randint(np.min(data), np.max(data), size = k)) # 랜덤하게 센터 잡음
    return np.array(centers).T

def mean_initialize(k, data,dim):
    centers = []
    size = len(data[0])//k
    i=0
    while True:
        centers.append(np.mean(data[:,i:i+size],axis=1)) ### 구간별로 나눠서 mean값으로 센터잡음
        i+=size
        if i+size>len(data[0]):
          break
    return centers

def eff_initialize(k, data, dim):
    visit = set()
    centers = np.zeros((k,dim))
    temp = copy.deepcopy(data.T)
    i = np.random.randint(0,temp.shape[0],size=1)
    i = int(i)
    visit.add(i)
    centers[0] = temp[i]
    while len(visit)<k:
        _next = np.argmax(((temp-temp[i])**2).sum(axis=1)) ### 거리가 가장 큰걸로 선택
        if _next not in visit: ### 방문 안했으면 선택
          centers[len(visit)] = temp[_next]
          visit.add(_next)
          i=_next
        else: ### 방문 했으면 거리 똑같이 해주고 다음꺼 찾음
          temp[_next] = temp[i]
    return centers
        

def kmeans(k, img, init='eff_initialize', distance_method = 'euclidean'):
    dim = 3
    data = img.reshape(-1,dim).T ### dim, -1
    
    ### 초기화 부분 ###
    if init=='random_initialize':
        centers = random_initialize(k,data,dim)
    elif init=='mean_initialize':
        centers = mean_initialize(k,data,dim)
    elif init=='eff_initialize':
        centers = eff_initialize(k,data,dim)
    pre_centers = copy.deepcopy(centers)
    ### 거리 메소드 선택 ###
    if distance_method == 'euclidean':
        method = euclidean
    elif distance_method == 'mahalanobis':
        method = mahalanobis
    
    
    count = 0
    while True:
        distances = []
        center_distance = {i:[] for i in range(k)}
        out = []
        for i,center in enumerate(centers):
            distance = method(data.T, center) # 원소개수, 센터개수으로 shape 나옴 -> 거리계산
            out.append(distance)
        distances = np.array(out)
        
        candidates = np.argmin(distances,axis=0) # 거리가 가장 작은걸로 해당 원소의 센터선택
        centers = []
        for _k in range(k):
            centers.append(data[:,np.where(candidates==_k)].mean(axis=2)) # 해당 센터에 포함되는 원소들의 mean값 계산
        centers = np.array(centers).squeeze()
        '''
        ### 차원 그림 출력하는 과정 주석처리 ###
        ### k가 여러개 있을 수 있으므로 색깔을 최대 8개로 만들었다 ###
        fig = plt.figure(figsize=(5, 5))
        ax = fig.gca(projection='3d')
        x,y,z = img.reshape(3,-1)
        x,y,z = data[:,np.where(candidates==0)]
        ax.scatter(x,y,z, marker='o', s=0.1, c='green', cmap='Greens')
        x,y,z = data[:,np.where(candidates==1)]
        ax.scatter(x,y,z, marker='o', s=0.1, c='yellow', cmap='Yellows')
        x,y,z = data[:,np.where(candidates==2)]
        ax.scatter(x,y,z, marker='o', s=0.1, c='blue', cmap='Blues')
        x,y,z = data[:,np.where(candidates==3)]
        ax.scatter(x,y,z, marker='o', s=0.1, c='red', cmap='Reds')
        x,y,z = data[:,np.where(candidates==4)]
        ax.scatter(x,y,z, marker='o', s=0.1, c='pink', cmap='Pinks')
        x,y,z = data[:,np.where(candidates==5)]
        ax.scatter(x,y,z, marker='o', s=0.1, c='cyan', cmap='Cyans')
        x,y,z = data[:,np.where(candidates==6)]
        ax.scatter(x,y,z, marker='o', s=0.1, c='purple', cmap='Purples')
        x,y,z = data[:,np.where(candidates==7)]
        ax.scatter(x,y,z, marker='o', s=0.1, c='orange', cmap='Oranges')
        
        for x,y,z in centers:
            ax.scatter(x,y,z, marker='o', s=10, c='darkred')
        plt.show()
        '''
        if np.isnan(centers).any(): # nan이면 break -> 실험할 예정
          #print('nan error')
          count = -1
          break
        
        if (pre_centers-centers).sum()==0: # 전과 동일하므로 break
            '''
            ### 이미지 출력하는 과정이므로 주석처리 ###
            result = np.zeros(img.shape)
            candidates = candidates.reshape(img.shape[0], img.shape[1])
            for h in range(result.shape[0]):
                for w in range(result.shape[1]):
                    result[h,w] = centers[candidates[h,w]]
            plt.imshow(result.astype('uint8'))
            plt.show()
            '''
            break
        pre_centers = copy.deepcopy(centers)
        count+=1 # while문 몇번 돌았는지 개수 세기 위해서
    return centers, count



### mean shift kernel 가중치 메소드 ###

def gaussian_calculate(x,y,h):
    k = np.linalg.norm(x-y)
    if k<=h:
        return np.exp(-(k**2))
    else:
        return 0
def average_calculate(x,y,h):
    k = np.linalg.norm(x-y)
    if k<=h:
        return 1
    else:
        return 0

### mean shift 초기화 메소드 ###
def mean_shift_center_initialize(radius, data, dim):
    centers = []
    visit = set()
    i = np.random.randint(0, data.shape[1],size=1)
    i = int(i)
    visit.add(i)
    while True:
      if len(visit)==data.shape[1]:
        break
      center = np.zeros((1,dim))
      center[0] = data[:,i]
      centers.append(data[:,i])
      distances = ((data.T-center)**2).sum(axis=1)
      
      indexes = set(np.where(distances<radius**2)[0].tolist())  ### 해당 센터로부터 raidus 안에 있으면 방문처리 
      
      visit = visit.union(indexes)
      
      for i, distance in enumerate(distances):
        if i not in visit:  ### 방문 안했으면 다음 센터로 지정(i 변수 그대로 이용한다)
          visit.add(i)
          break
    return np.array(centers).reshape(dim,-1)
      
      


def mean_shift(radius, img, filtermode='average'):
    data = img.reshape(-1,3).T.astype('float32')
    
    dim = data.shape[0]
    #centers = copy.deepcopy(data) # 모든 픽셀들을 센터로 잡음
    centers = mean_shift_center_initialize(radius, data, dim) # 내가 만든 초기화 함수로 센터 잡음
    pre_centers = copy.deepcopy(centers)
    if filtermode=='gaussian':
        kernel = gaussian_calculate
    elif filtermode=='average':
        kernel = average_calculate
        
    segmentation_img = np.zeros(data.shape) # segmentation 할 이미지 생성
    while True:

        '''
        ### plot 과정이므로 주석처리 함 ###
        ## 3차원으로 scatter ##
        fig = plt.figure(figsize=(5, 5))
        ax = fig.gca(projection='3d')
        x,y,z = data
        ax.scatter(x,y,z, marker='o', s=0.1, c='green', cmap='Greens', alpha=0.3)
        
        x,y,z = centers
        ax.scatter(x, y, z, facecolor=(0,0,0,0), s=np.pi*(radius**2), marker='o', edgecolor='red')
        plt.show()
        '''
        print('Center개수 : ', centers.shape[1])
        
        new_centers = []
        
        
        for center in centers.T:
            sum1=0
            sum2=0
            
            indexes = np.where(np.linalg.norm(data.T-center,axis=1)<radius) ### 거리안에 있는 원소들의 인덱스 추출
            
            for index in indexes[0]: ### 가중치 계산과정
                x = data[:,index].T
                
                y = center
                h = radius
                k = kernel(x,y,h)
                
                sum1 += (x*k)
                sum2 += k
            if sum2!=0:
                new_centers.append(sum1/sum2)
                for index in indexes[0]:
                  segmentation_img[:,index]=sum1/sum2 ### 가중치 계산했으므로 나눠줘야됌
        new_centers = np.array(new_centers)
        
        new_centers = np.unique(new_centers,axis=0) ### 센터 중복 제거
        new_centers = new_centers.T
        if centers.shape==new_centers.shape and (centers-new_centers).sum()<=1: ### 센터 크기 같고, 뺀 값들의 sum이 1보다 작을때 break, 0으로 해도 수렴하긴 함. 그러나 오래걸림
            break
        
        centers = copy.deepcopy(new_centers)

    '''
    ### 최종 segmentation 그림 출력 과정 ###
    segmentation_img = segmentation_img.T
    segmentation_img = segmentation_img.reshape(img.shape[0], img.shape[1], dim)
    plt.imshow(segmentation_img.astype('uint8'))
    plt.show()
    '''
    return centers
        


In [60]:


### kmeans ###
path = 'tt.jpg'
img = cv2.imread(path, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = np.array(img)
k=3
kmeans(k, img, init='eff_initialize', distance_method='euclidean')
# center와 count로 나온다

(array([[227.21988174, 163.65225425, 148.390983  ],
        [109.38152914,  35.02535958,  67.87887964],
        [192.76268272,  97.63413586, 101.56319862]]), 32)

In [61]:
'''
### 성능 비교 ###
### 실험 결과는 레포트에 작성함 ###
### 너무 오래 걸려서 주석처리 ###
path = 'tt.jpg'
img = cv2.imread(path, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = np.array(img)
k=3

random_euclidean = []
random_mahalanobis = []
mean_euclidean = []
mean_mahalanobis = []
eff_euclidean = []
eff_mahalanobis = []
for _ in range(100):
  _, count = kmeans(k, img, init='random_initialize', distance_method='euclidean')
  random_euclidean.append(count)
  _, count = kmeans(k, img, init='random_initialize', distance_method='mahalanobis')
  random_mahalanobis.append(count)
  _, count = kmeans(k, img, init='mean_initialize', distance_method='euclidean')
  mean_euclidean.append(count)
  _, count = kmeans(k, img, init='mean_initialize', distance_method='mahalanobis')
  mean_mahalanobis.append(count)
  _, count = kmeans(k, img, init='eff_initialize', distance_method='mahalanobis')
  eff_euclidean.append(count)
  _, count = kmeans(k, img, init='eff_initialize', distance_method='mahalanobis')
  eff_mahalanobis.append(count)
'''

"\n### 성능 비교 ###\n### 실험 결과는 레포트에 작성함 ###\n### 너무 오래 걸려서 주석처리 ###\npath = 'tt.jpg'\nimg = cv2.imread(path, cv2.IMREAD_COLOR)\nimg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)\nimg = np.array(img)\nk=3\n\nrandom_euclidean = []\nrandom_mahalanobis = []\nmean_euclidean = []\nmean_mahalanobis = []\neff_euclidean = []\neff_mahalanobis = []\nfor _ in range(100):\n  _, count = kmeans(k, img, init='random_initialize', distance_method='euclidean')\n  random_euclidean.append(count)\n  _, count = kmeans(k, img, init='random_initialize', distance_method='mahalanobis')\n  random_mahalanobis.append(count)\n  _, count = kmeans(k, img, init='mean_initialize', distance_method='euclidean')\n  mean_euclidean.append(count)\n  _, count = kmeans(k, img, init='mean_initialize', distance_method='mahalanobis')\n  mean_mahalanobis.append(count)\n  _, count = kmeans(k, img, init='eff_initialize', distance_method='mahalanobis')\n  eff_euclidean.append(count)\n  _, count = kmeans(k, img, init='eff_initialize', distance_

In [62]:
### mean_shift ###
import time


path = 'tt.jpg'
img = cv2.imread(path, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = np.array(img)
mean_shift(50, img, filtermode='average')
### mean shift 실험 과정 ###
### 레포트에 작성했으므로 주석처리 ###
'''
### average vs gaussian ###
start = time.time()
mean_shift(30,img)
print("average 적용했을때 시간 : {}".format(time.time()-start))

start = time.time()
mean_shift(30,img,filtermode='gaussian')
print("gaussian 적용했을때 시간 : {}".format(time.time()-start))
'''
'''
### window size 비교 ###
start = time.time()
mean_shift(30,img)
print("average 적용했을때 시간 : {}".format(time.time()-start))
start = time.time()
mean_shift(50,img)
print("average 적용했을때 시간 : {}".format(time.time()-start))
start = time.time()
mean_shift(100,img)
print("average 적용했을때 시간 : {}".format(time.time()-start))
'''

Center개수 :  13
Center개수 :  7
Center개수 :  7
Center개수 :  7


'\n### window size 비교 ###\nstart = time.time()\nmean_shift(30,img)\nprint("average 적용했을때 시간 : {}".format(time.time()-start))\nstart = time.time()\nmean_shift(50,img)\nprint("average 적용했을때 시간 : {}".format(time.time()-start))\nstart = time.time()\nmean_shift(100,img)\nprint("average 적용했을때 시간 : {}".format(time.time()-start))\n'